### Funktionen alle paar Millisekunden  wieder aufrufen
Programmiert man ein Spiel, so m&ouml;chte man z.B. alle paar Millisekunden die Leinwand updaten. 

Nachstehend importieren wir ein Widget, welches 
eine Leinwand, ein Output-Widget und Buttones enth&auml;lt. 
Der `clear`-Button l&ouml;scht die Leinwand und das Output-Widget.
Die anderen Buttons geben aus, dass sie gedr&uuml;ckt wurden. 
Sind `start`, `stop` und `pause` Funktionen, so k&ouml;nnen diese mit  
`widget.register_callbacks({'start': start, 'pause': pause, 'stop': stop})` als
Callbacks f&uuml;r die Buttons registriert werden.

`widget.draw_pt()` malt einen zuf&auml;lligen Punkt auf die Leinwand.  
Ist die Leinwand im Fokus, so kann durch Dr&uuml;cken von 'b','g', oder 'r' die Farbe
gewechselt werden.

In [ ]:
from widget_helpers import Widget

In [ ]:
widget = Widget()
widget

In [ ]:
widget.draw_pt()

### Der naive Ansatz: While-Loop und `time.sleep`
Nachstehender Code zeichnet alle 0.1 Sekunden einen Punkt auf die Leinwand.  
**Aber**: Der While-Loop blockiert das Notebook und die Buttons funktioniern nicht mehr.

In [ ]:
import time
# zeichet alle 0.1 Sekunden einen neuen Punkt
# aber blockiert das Notebook!
# Buttons funktionieren nicht mehr!
def loop():
    while True:
        widget.draw_pt()
        time.sleep(0.1)

In [ ]:
loop()

In [ ]:
print('test')

### Variante mit asyncio
Wir definieren eine async Coroutine
```python
async def loop_async():  
    while state['running']:
        
        await state['resume'].wait()
        widget.draw_pt()
        do_stuff()
        await asyncio.sleep(state['delay'])
```
und f&uuml;gen diese als Task in den aktuellen Eventloop ein und
starten den Task.

```python
def start_loop_async():
    loop = asyncio.get_event_loop()
    task = loop.create_task(loop_async())
    loop.call_soon_threadsafe(asyncio.ensure_future, task)
```

In [ ]:
import asyncio

state = {'running': True,
         'delay': 1,
         'pause': False,
         'count': 0
        }

# @widget.out.capture() # (should work but doesn't)
def do_stuff():
    state['count'] += 1
    msg = 'running: {}, pause: {}, count:{}\n'.\
          format(state['running'],  state['pause'], state['count'])
    widget.out.append_stdout(msg)    
                 
async def loop_async():  
    while state['running']:
        
        if not state['pause']:
            widget.draw_pt()
            do_stuff()
        await asyncio.sleep(state['delay'])
        
def start_loop_async():
    loop = asyncio.get_event_loop()
    task = loop.create_task(loop_async())
    loop.call_soon_threadsafe(asyncio.ensure_future, task)
      
def start():
    print('start loop')
    state['running'] = True
    state['pause'] = False
    start_loop_async()
    
def pause():
    state['pause'] = not state['pause'] 
    print('toggled pause')
    
def stop():
    print('stop loop')
    state['running'] = False
    
callbacks = {'start': start, 'pause': pause, 'stop': stop}

In [ ]:
widget.register_callbacks(callbacks)
widget

### Variante with Threads  
Die Funktion `def start_loop_thread()` macht sich zu einem Thread, der sich selber nach einem kurzen Delay wieder started.

In [ ]:
import threading

# Thread Variante
def start_loop_thread():  
    if state['running']:
        if not state['pause']:
            widget.draw_pt()
            do_stuff()
        
        t = threading.Timer(state['delay'], start_loop_thread, args=[])
        t.start()

def start():
    print('start loop')
    state['running'] = True
    state['pause'] = False
    start_loop_thread()

# update callback fuer start-Button
callbacks = {'start': start}

In [ ]:
widget.register_callbacks(callbacks)
widget